<a href="https://colab.research.google.com/github/mlupu98/DataRes-Sports-F19/blob/master/NBA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
df = pd.read_csv("nba.csv")
df1 = df.drop_duplicates(['teamAbbr','teamFTA', 'teamFTM', 'opptAbbr', 'opptFTA', 'opptFTM'], keep= 'last')
#df1 = df1[df1.index % 2 == 0]
df1.head()

,offLNm,offFNm,teamAbbr,teamLoc,teamRslt,teamFTA,teamFTM,teamFT%,opptAbbr,opptConf,opptLoc,opptRslt,opptFTA,opptFTM,opptFT%
2,Workman,Haywoode,WAS,Away,Loss,20,12,0.6000,CLE,East,Home,Win,22,15,0.6818
5,Workman,Haywoode,CLE,Home,Win,22,15,0.6818,WAS,East,Away,Loss,20,12,0.6000
8,Fitzgerald,Kane,BOS,Away,Loss,28,23,0.8214,MIA,East,Home,Win,32,26,0.8125
11,Fitzgerald,Kane,MIA,Home,Win,32,26,0.8125,BOS,East,Away,Loss,28,23,0.8214
14,Dalen,Eric,DAL,Away,Win,18,14,0.7778,LAL,West,Home,Loss,31,12,0.3871


In [0]:
teams = df1.teamAbbr.unique()
teams

array(['WAS', 'CLE', 'BOS', 'MIA', 'DAL', 'LAL', 'DEN', 'PHI', 'IND',
       'TOR', 'HOU', 'DET', 'SAC', 'CHI', 'SA', 'NO', 'UTA', 'GS', 'PHO',
       'MEM', 'LAC', 'POR', 'OKC', 'ORL', 'CHA', 'MIL', 'ATL', 'NY',
       'MIN', 'BKN'], dtype=object)

In [0]:
column = ','.join(map(str, teams))
column = 'Team,Home,Away,Combined,Difference,' + column
column = column.split(",")
result = pd.DataFrame(index = range(teams.size), columns = column)
result.head()
#%pprint
#column

,Team,Home,Away,Combined,Difference,WAS,CLE,BOS,MIA,DAL,...,LAC,POR,OKC,ORL,CHA,MIL,ATL,NY,MIN,BKN
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
idx = 0
while (idx < teams.size):
    home = df1[((df1['teamAbbr'] == teams[idx]) & (df1['teamLoc'] == 'Home'))]
    away = df1[((df1['teamAbbr'] == teams[idx]) & (df1['teamLoc'] == 'Away'))]
    combined = pd.concat([home, away])
    result['Team'].iloc[idx] = teams[idx]
    result['Home'].iloc[idx] = home['teamFT%'].mean()
    result['Away'].iloc[idx] = away['teamFT%'].mean()
    result['Combined'].iloc[idx] = combined['teamFT%'].mean()
    idx = idx + 1
result.head()

,Team,Home,Away,Combined,Difference,WAS,CLE,BOS,MIA,DAL,...,LAC,POR,OKC,ORL,CHA,MIL,ATL,NY,MIN,BKN
0,WAS,0.762397,0.739296,0.750847,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CLE,0.749017,0.757937,0.753477,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BOS,0.780503,0.782491,0.781499,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MIA,0.75118,0.740734,0.745957,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DAL,0.787156,0.776328,0.781742,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
idx = 0
while (idx < result.shape[0]):
    idx2 = 0
    home = df1[((df1['teamAbbr'] == teams[idx]) & (df1['teamLoc'] == 'Home'))]
    away = df1[((df1['teamAbbr'] == teams[idx]) & (df1['teamLoc'] == 'Away'))]
    #free throw % playing away in this team's home court - league average away free throw % excluding this team
    result2 = result[result['Team'] != teams[idx]]
    result['Difference'].iloc[idx] = home['opptFT%'].mean() - result2['Away'].mean()
    #free throw % playing away at other teams' home courts
    while (idx2 < result.shape[0]):
        if (teams[idx2] != teams[idx]):
            result[teams[idx2]].iloc[idx] = (away[away['opptAbbr'] == teams[idx2]])['teamFT%'].mean()
        idx2 = idx2 + 1
    idx = idx + 1
result.head()

,Team,Home,Away,Combined,Difference,WAS,CLE,BOS,MIA,DAL,...,LAC,POR,OKC,ORL,CHA,MIL,ATL,NY,MIN,BKN
0,WAS,0.762397,0.739296,0.750847,-0.00328066,NaN,0.7811,0.739491,0.720425,0.74635,...,0.64145,0.719433,0.70465,0.728867,0.77045,0.744827,0.76035,0.70086,0.767283,0.723309
1,CLE,0.749017,0.757937,0.753477,0.000225954,0.807555,NaN,0.782282,0.7365,0.758617,...,0.694933,0.692517,0.779283,0.7607,0.779891,0.784367,0.728127,0.7492,0.7473,0.753655
2,BOS,0.780503,0.782491,0.781499,-0.00176363,0.781045,0.80144,NaN,0.825482,0.830283,...,0.812367,0.787133,0.645633,0.786309,0.78515,0.787609,0.787073,0.790675,0.767317,0.767883
3,MIA,0.75118,0.740734,0.745957,0.00160833,0.750233,0.745064,0.72074,NaN,0.711783,...,0.77865,0.733383,0.7901,0.755242,0.75365,0.76978,0.785975,0.732955,0.750383,0.764445
4,DAL,0.787156,0.776328,0.781742,-0.00643052,0.805183,0.820733,0.797483,0.74275,NaN,...,0.7707,0.83326,0.776873,0.70935,0.777017,0.814183,0.828383,0.85805,0.734682,0.7916


In [0]:
result.to_csv('freethrows.csv')
result.head()

,Team,Home,Away,Combined,Difference,WAS,CLE,BOS,MIA,DAL,...,LAC,POR,OKC,ORL,CHA,MIL,ATL,NY,MIN,BKN
0,WAS,0.762397,0.739296,0.750847,-0.00328066,NaN,0.7811,0.739491,0.720425,0.74635,...,0.64145,0.719433,0.70465,0.728867,0.77045,0.744827,0.76035,0.70086,0.767283,0.723309
1,CLE,0.749017,0.757937,0.753477,0.000225954,0.807555,NaN,0.782282,0.7365,0.758617,...,0.694933,0.692517,0.779283,0.7607,0.779891,0.784367,0.728127,0.7492,0.7473,0.753655
2,BOS,0.780503,0.782491,0.781499,-0.00176363,0.781045,0.80144,NaN,0.825482,0.830283,...,0.812367,0.787133,0.645633,0.786309,0.78515,0.787609,0.787073,0.790675,0.767317,0.767883
3,MIA,0.75118,0.740734,0.745957,0.00160833,0.750233,0.745064,0.72074,NaN,0.711783,...,0.77865,0.733383,0.7901,0.755242,0.75365,0.76978,0.785975,0.732955,0.750383,0.764445
4,DAL,0.787156,0.776328,0.781742,-0.00643052,0.805183,0.820733,0.797483,0.74275,NaN,...,0.7707,0.83326,0.776873,0.70935,0.777017,0.814183,0.828383,0.85805,0.734682,0.7916
